In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import glob
import os
videos = glob.glob("./gdrive/MyDrive/KiMoRe/RGB/**/*.mp4", recursive=True)
counter = 0
Ex_type = "Es1"
for video_path in videos:
    if Ex_type not in video_path: 
        continue

    new_name = os.path.join(*(video_path.split("/")[5:]))
    new_name = new_name.replace("/", "_")
    new_name = new_name.split(".")[0]
    # Run openpose on uploaded video and save output mp4 video
    folder_name = "./gdrive/MyDrive/KiMoRe_skeletal_" + Ex_type + "/" + new_name

    # Analyze video on Google Drive (The analysis result "openpose.avi" is output to the root directory of Google Drive)
    # --disable_blending set to true to render on a blackground
    ! cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 1 --video ../"$video_path" --display 0 --disable_blending 1 --write_json ../"$folder_name"/ --write_video ../"$folder_name"/openpose.avi

    # config: --keypoint_scale set to 3 to scale body joints to the range [0,1], where (0,0) would be the top-left"
    # ! cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 1 --video ../"$video_path" --display 0 --disable_blending 1 --write_json "$folder_name" --keypoint_scale 3


Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 32.803899 seconds.
1
./gdrive/MyDrive/KiMoRe_skeletal_Es5/CG_Expert_E_ID10_Es5_rgb_Blur_rgb011214_104814
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 22.007230 seconds.
2
./gdrive/MyDrive/KiMoRe_skeletal_Es5/CG_Expert_E_ID11_Es5_rgb_Blur_rgb011214_105314
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 27.481445 seconds.
3
./gdrive/MyDrive/KiMoRe_skeletal_Es5/CG_Expert_E_ID12_Es5_rgb_Blur_rgb011214_105934
Starting OpenPose demo...
Configuring OpenPose...

In [ ]:
!pwd
# !ffmpeg -y -loglevel info -i openpose/output_video/openpose.avi openpose/output_video/output.mp4

def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video("openpose/output_video/output.mp4", width=960, height=720)